In [58]:
from utils import read_csv
from utils import *
relevant_tokens = [
    "CommandArgument",
    "Command",
    "CommandParameter",
    "Member"
]


def functional_level(tokens_filepath):
    data = read_csv(tokens_filepath)
    data = data[1:]
    output = {}
    for row in data:
        if row[1] in relevant_tokens:
            if output.get(row[1]) == None:
                output[row[1]] = {}
            if output[row[1]].get(row[0]) == None:
                output[row[1]][row[0]] = 1
            else:
                output[row[1]][row[0]] = output[row[1]][row[0]]+1
    return output


In [59]:
import os

def fldir(dir):
    dir_data = {}
    for file in os.listdir(dir):
        file_data = functional_level(os.path.join(dir,file))
        for t in relevant_tokens:
            if t not in file_data:
                continue
            for key,val in file_data[t].items():
                if t not in dir_data:
                    dir_data[t] = {}
                if key not in dir_data[t]:
                    dir_data[t][key] = 0
                dir_data[t][key] = dir_data[t][key] + file_data[t][key]
    return dir_data

benign_token_data = fldir('../tokens/benign')
malicious_token_data = fldir('../tokens/malicious/')



In [60]:
for t in relevant_tokens:
    benign_token_data[t] = sort_dict_by_value(benign_token_data[t])
    malicious_token_data[t] = sort_dict_by_value(malicious_token_data[t])

In [61]:
total_benign_tokens = 0
total_malicious_tokens = 0
for t in relevant_tokens:
    total_benign_tokens += sum(benign_token_data[t].values())
    total_malicious_tokens += sum(malicious_token_data[t].values())
total_malicious_tokens

132200

In [62]:
token_count = {}
for t in relevant_tokens:
    malicious_dict = malicious_token_data[t]
    benign_dict = benign_token_data[t]
    # union of keys in both dictionaries
    for key in set(malicious_dict.keys()).union(set(benign_dict.keys())):
        mc = 0
        bc = 0
        if key in malicious_dict:
            mc = malicious_dict[key]
        if key in benign_dict:
            bc = benign_dict[key]
        mc = mc/total_malicious_tokens
        bc = bc/total_benign_tokens
        token_count[key] = {"malicous":mc,"benign":bc,"total":mc+bc,"score":mc-bc}

def sort_dict_by_value_field(d,field):
    return OrderedDict(sorted(d.items(), key=lambda x: x[1][field], reverse=True))

token_count = sort_dict_by_value_field(token_count,"score")
token_count




OrderedDict([('Length',
              {'malicous': 0.055885022692889565,
               'benign': 0.004115023421565771,
               'total': 0.060000046114455335,
               'score': 0.051769999271323794}),
             ('Out-Null',
              {'malicous': 0.036059001512859304,
               'benign': 0.004776953478121194,
               'total': 0.0408359549909805,
               'score': 0.03128204803473811}),
             ('Get-Random',
              {'malicous': 0.023774583963691375,
               'benign': 0.0002525195926814144,
               'total': 0.02402710355637279,
               'score': 0.02352206437100996}),
             ('DefineField',
              {'malicous': 0.02542360060514372,
               'benign': 0.001982801772119745,
               'total': 0.027406402377263467,
               'score': 0.023440798833023975}),
             ('-Input',
              {'malicous': 0.019652042360060514,
               'benign': 0.0,
               'total': 0.019652042

In [63]:
def file_score(tokens_filepath):
    data = read_csv(tokens_filepath)
    data = data[1:]
    score = 0
    for row in data:
        if row[0] in token_count:
            score = score + token_count[row[0]]["score"]
    return score

count= 0 
for file in os.listdir('../tokens/malicious/'):
    if file_score(os.path.join('../tokens/malicious',file)) > 0:
        count = count + 1

In [64]:
count

4157

In [65]:
count= 0 
for file in os.listdir('../tokens/benign/'):
    if file_score(os.path.join('../tokens/benign',file)) < 0:
        count = count + 1

In [66]:
count

3243

In [67]:
print(len(os.listdir('../tokens/benign/')))

4316
